

# Moroccan Darija Language Model using fastai

In [ ]:
!pip install fastai -q --upgrade 
!pip install -q sentencepiece!=0.1.90,!=0.1.91 

     |████████████████████████████████| 186 kB 5.4 MB/s 
     |████████████████████████████████| 56 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [ ]:
import sys
from gensim.corpora import WikiCorpus
from fastai.text.all import *
import torch as torch
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
path = Path('/content/drive/MyDrive/ml/projects/darija/')
dls_path = path/'dls'
model_path = path/'models'
spm_path = model_path/'spm'

dls_path.mkdir(exist_ok=True, parents=True)
model_path.mkdir(exist_ok=True, parents=True)
spm_path.mkdir(exist_ok=True, parents=True)

Moroccan Darija Wikipedia data, available in this [link](https://dumps.wikimedia.org/arywiki/20210520/).

In [ ]:
!wget https://dumps.wikimedia.org/arywiki/latest/arywiki-latest-pages-articles.xml.bz2 -O '/content/drive/MyDrive/ml/projects/darija/arywiki-latest-pages-articles.xml.bz2'

--2021-09-15 10:00:11--  https://dumps.wikimedia.org/arywiki/latest/arywiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4346202 (4.1M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/ml/projects/darija/arywiki-latest-pages-articles.xml.bz2’

/content/drive/MyDr 100%[===================>]   4.14M  4.75MB/s    in 0.9s    

2021-09-15 10:00:12 (4.75 MB/s) - ‘/content/drive/MyDrive/ml/projects/darija/arywiki-latest-pages-articles.xml.bz2’ saved [4346202/4346202]



We make use of [WikiCorpus](https://radimrehurek.com/gensim/corpora/wikicorpus.html) from gensim to convert the XML file we downloaded to a text corpus. 



In [ ]:
def make_corpus(in_f, out_f):

  """Convert Wikipedia xml dump file to text corpus"""

  output = open(out_f, 'w')
  wiki = WikiCorpus(in_f)

  for i, text in enumerate(wiki.get_texts()):
    output.write(bytes(' '.join(text), 'utf-8').decode('utf-8') + '\n')

    if (i % 1000 == 0):
      print('Processed ' + str(i) + ' articles')
    
  output.close()
  print('Processing complete!')


make_corpus(f'{path}/arywiki-latest-pages-articles.xml.bz2', f'{path}/wiki_darija.txt')

Processed 0 articles
Processed 1000 articles
Processed 2000 articles
Processed 3000 articles
Processing complete!


In [ ]:
path.ls()

(#4) [Path('/content/drive/MyDrive/ml/projects/darija/arywiki-latest-pages-articles.xml.bz2'),Path('/content/drive/MyDrive/ml/projects/darija/dls'),Path('/content/drive/MyDrive/ml/projects/darija/models'),Path('/content/drive/MyDrive/ml/projects/darija/wiki_darija.txt')]

In [ ]:
df = pd.read_csv(path/'wiki_darija.txt', header=None, names=['text'])
df.head()

,text
0,آبطح جماعة ترابية قروية كاينة إقليم عمالة طان طان جهة ݣلميم واد نون ساكنين فيها واحد على حسب لإحصاء لعام تعليم نسبة لأمية اس ما كايعرفوش يقراو ولا يكتبو نسبة كان قاريين فوق انوي تانوي جامعة اقتصاد نسبة اس شيطين يقدرو يخدمو نسبة لبطالة اس ما خدامينش تايقلبو على خدمة نسبة اس اللي خدامين ولة ولا لعاطلين اللي سبق ليهوم خدمو نسبة اس اللي خدامين في لقطاع لخاص ولا لعاطلين اللي سبق ليهوم خدمو عيون لكلام تصنيف جهة ݣلميم واد نون
1,آسفي بالأمازيغية ⴰⵙⴼⵉ هي مدينة مغربية جات إقليم آسفي جهة مراكش آسفي آسفي معروفة بالفخار والحوت وخصوصا السردين ومكنيين عليها حاضرة المحيط الحطة ديال آسفي جات كاطل على المحيط الأطلسي بين الجديدة والصويرة في آسفي كاين بزاف دالبني لي قديم وتاريخي وهي من بين المدون القديمة في المغرب ساكنين فيها واحد على حسب لإحصاء لعام تعليم نسبة لأمية اس ما كايعرفوش يقراو ولا يكتبو نسبة كان قاريين فوق انوي تانوي جامعة اقتصاد نسبة اس شيطين يقدرو يخدمو نسبة لبطالة اس ما خدامينش تايقلبو على خدمة نسبة اس اللي خدامين ولة ولا لعاطلين اللي سبق ليهوم خدمو نسبة اس اللي خدامين في لقطاع لخاص ولا لعاطلين اللي سبق ليهوم خد...
2,آلبرخت دورر بالألمانية albrecht dürer ماي أبريل رسام صانع طباعة وم كاينتامي لعصر النهضة الألمانية تزاد في نورمبرݣ دورر أسس للسمعة والتأثير ديالو عبر أوروبا فالوقت اللي مازالا فالعشرينات من عمرو نتيجة لجودة نقوشاتو الخشبية كان فاتصال مع أكبر الفنانين الإيطاليين فالعصر ديالو بما فيهم رفائيل جيوڤاني بيليني وليوناردو دا ڤينتشي وابتداء من كان كي اخد الدعم من عند الإمبراطور ماكسيميليان الأول تم تشييع دورر فالكنيستين اللوثرية والأسقفية بجوج كتشمل مجموعة أعمال دورر الواسعة النقوش أسلوبو المفضل في طبعاتو الأخيرة الأعمال الفنية فالكنائس altarpieces پورتريهات پورتريهات ذاتية لوحات مائية وك النقوش الخ...
3,آمتدي جماعة ترابية قروية كاينة إقليم عمالة ݣلميم جهة ݣلميم واد نون ساكنين فيها واحد على حسب لإحصاء لعام تعليم نسبة لأمية اس ما كايعرفوش يقراو ولا يكتبو نسبة كان قاريين فوق انوي تانوي جامعة اقتصاد نسبة اس شيطين يقدرو يخدمو نسبة لبطالة اس ما خدامينش تايقلبو على خدمة نسبة اس اللي خدامين ولة ولا لعاطلين اللي سبق ليهوم خدمو نسبة اس اللي خدامين في لقطاع لخاص ولا لعاطلين اللي سبق ليهوم خدمو عيون لكلام تصنيف جهة ݣلميم واد نون
4,آنفݣ جماعة ترابية قروية كاينة إقليم عمالة سيدي إيفني جهة ݣلميم واد نون ساكنين فيها واحد على حسب لإحصاء لعام تعليم نسبة لأمية اس ما كايعرفوش يقراو ولا يكتبو نسبة كان قاريين فوق انوي تانوي جامعة اقتصاد نسبة اس شيطين يقدرو يخدمو نسبة لبطالة اس ما خدامينش تايقلبو على خدمة نسبة اس اللي خدامين ولة ولا لعاطلين اللي سبق ليهوم خدمو نسبة اس اللي خدامين في لقطاع لخاص ولا لعاطلين اللي سبق ليهوم خدمو عيون لكلام تصنيف جهة ݣلميم واد نون


In [ ]:
bs=128

tok = SubwordTokenizer(cache_dir=spm_path, max_vocab_sz=1000)

dls_lm = DataBlock(blocks=TextBlock.from_df('text', is_lm=True, tok=tok),
                   splitter=RandomSplitter(0.1, seed=42),
                   get_x=ColReader('text')
                   ).dataloaders(df, bs=bs)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
torch.save(dls_lm, dls_path/'dls_lm.pkl')

In [ ]:
dls_lm.show_batch(max_n=6)

,text,text_
0,▁xxbos ▁ولا د ▁ع يس ى ▁جماع ة ▁ترابية ▁قرو ية ▁كاين ة ▁إقليم ▁عمال ة ▁ خر يب ݣ ة ▁جه ة ▁بني ▁ ملال ▁خنيفرة ▁ساكنين ▁فيها ▁واحد ▁عل ى ▁حسب ▁ل إحصاء ▁لعام ▁تعليم ▁نسبة ▁لأمي ة ▁اس ▁ما ▁كايعرفو ش ▁يقرا و ▁ولا ▁يكتب و ▁نسبة ▁كان ▁ قاريين ▁فوق ▁ انوي ▁تانوي ▁جامع ة ▁اقتصاد ▁نسبة ▁اس ▁شيط ين ▁ يقدرو ▁ يخدمو ▁نسبة ▁لبطال ة ▁اس,▁ولا د ▁ع يس ى ▁جماع ة ▁ترابية ▁قرو ية ▁كاين ة ▁إقليم ▁عمال ة ▁ خر يب ݣ ة ▁جه ة ▁بني ▁ ملال ▁خنيفرة ▁ساكنين ▁فيها ▁واحد ▁عل ى ▁حسب ▁ل إحصاء ▁لعام ▁تعليم ▁نسبة ▁لأمي ة ▁اس ▁ما ▁كايعرفو ش ▁يقرا و ▁ولا ▁يكتب و ▁نسبة ▁كان ▁ قاريين ▁فوق ▁ انوي ▁تانوي ▁جامع ة ▁اقتصاد ▁نسبة ▁اس ▁شيط ين ▁ يقدرو ▁ يخدمو ▁نسبة ▁لبطال ة ▁اس ▁ما
1,ة ▁كات قص ر ▁ف ـ ▁بو طو لا ▁ت أ س س ات ▁ف ـ ▁كت لعب ▁فال ت ير ان ▁ديال ▁ال ش ه يد ▁أ ح م د ▁ ش ك ري ▁اللي ▁كي ه ز ▁م ش ج ع ▁لم در يب ▁ديال ▁لفرق ة ▁هو ▁س ع يد ▁شي ب ة ▁د ات ▁هاد ▁ال فر ق ة ▁ لبو طو لا ▁لوطني ة ▁لح ت ير,▁كات قص ر ▁ف ـ ▁بو طو لا ▁ت أ س س ات ▁ف ـ ▁كت لعب ▁فال ت ير ان ▁ديال ▁ال ش ه يد ▁أ ح م د ▁ ش ك ري ▁اللي ▁كي ه ز ▁م ش ج ع ▁لم در يب ▁ديال ▁لفرق ة ▁هو ▁س ع يد ▁شي ب ة ▁د ات ▁هاد ▁ال فر ق ة ▁ لبو طو لا ▁لوطني ة ▁لح ت ير افي
2,ية ▁ ش اد ▁الشمال ▁الشرق ي ▁ديال ▁ؤ س طر الي ا ▁كان و ▁السكان ▁لأ ص لي ين ▁د ؤ س طر الي ا ▁ هوما ▁اللي ▁جا و ▁لي ها ▁ال لو لين ▁ما ب ين ▁هاد ي ▁ آ لف ▁عام ▁ول ▁ ات ▁من ▁بعد ▁تا بع ة ▁لل ت اج ▁دل م م ل ك ة ▁لم ت حد ة ▁من ▁مورا ▁مات فر ق ات,▁ ش اد ▁الشمال ▁الشرق ي ▁ديال ▁ؤ س طر الي ا ▁كان و ▁السكان ▁لأ ص لي ين ▁د ؤ س طر الي ا ▁ هوما ▁اللي ▁جا و ▁لي ها ▁ال لو لين ▁ما ب ين ▁هاد ي ▁ آ لف ▁عام ▁ول ▁ ات ▁من ▁بعد ▁تا بع ة ▁لل ت اج ▁دل م م ل ك ة ▁لم ت حد ة ▁من ▁مورا ▁مات فر ق ات ▁عل
3,ضر وري ▁ن ف س ▁لك و اي ري ▁ي كون ▁ح ار س ▁فال ت ير ات ▁الت ر جي ح ▁حتى ▁شي ▁كو اي ري ▁ما ع ند و ▁لح ق ▁ي ع ا ود ▁ت ير ▁ آ خ ور ▁حتى ▁ يد وز و ▁ صح اب و ▁كا م لين ▁وم ع ا هوم ▁لح ار س ▁فاش ▁كاي ول ▁ ضر وري ▁باش ▁لك و ا,وري ▁ن ف س ▁لك و اي ري ▁ي كون ▁ح ار س ▁فال ت ير ات ▁الت ر جي ح ▁حتى ▁شي ▁كو اي ري ▁ما ع ند و ▁لح ق ▁ي ع ا ود ▁ت ير ▁ آ خ ور ▁حتى ▁ يد وز و ▁ صح اب و ▁كا م لين ▁وم ع ا هوم ▁لح ار س ▁فاش ▁كاي ول ▁ ضر وري ▁باش ▁لك و ا ير
4,ي و اء ▁ وف اش ▁ال ل رض ▁اللي ▁كاي دو ز ▁من ها ▁خ ط ▁اللي ل ▁كات كون ▁م س ط ▁ح ة ▁م كا ين ين ش ▁ع و ا يق ▁بحال ▁ال ج ب ال ▁كات وصل ▁ال ز ر ب ة ▁ديال ▁هاد ▁لخ ط ▁م يت رو ▁فال ت اني ة ▁هاد ▁الس ر ع ة ▁ت قدر ▁ت ب ان ▁باللي ▁كات ز اد,و اء ▁ وف اش ▁ال ل رض ▁اللي ▁كاي دو ز ▁من ها ▁خ ط ▁اللي ل ▁كات كون ▁م س ط ▁ح ة ▁م كا ين ين ش ▁ع و ا يق ▁بحال ▁ال ج ب ال ▁كات وصل ▁ال ز ر ب ة ▁ديال ▁هاد ▁لخ ط ▁م يت رو ▁فال ت اني ة ▁هاد ▁الس ر ع ة ▁ت قدر ▁ت ب ان ▁باللي ▁كات ز اد ▁فاش
5,دا ▁هي ▁مدين ة ▁أ تر ية ▁ قد يم ة ▁بن ا و ها ▁لم ور ▁لأ ما زي غ ▁لق ر ن ▁تال ت ▁قبل ▁لم يل اد ▁كانت ▁عندها ▁عل اق ات ▁ت ج اري ة ▁معا ▁ قر طا ج ▁لم و ق ع ▁ديال ▁هاد ▁لمدين ة ▁كاين ▁ح دا ▁تطوان ▁غ ال با ▁بن ا و ها ▁كان ▁ديال ▁تا م و دا ▁ ها ج,▁هي ▁مدين ة ▁أ تر ية ▁ قد يم ة ▁بن ا و ها ▁لم ور ▁لأ ما زي غ ▁لق ر ن ▁تال ت ▁قبل ▁لم يل اد ▁كانت ▁عندها ▁عل اق ات ▁ت ج اري ة ▁معا ▁ قر طا ج ▁لم و ق ع ▁ديال ▁هاد ▁لمدين ة ▁كاين ▁ح دا ▁تطوان ▁غ ال با ▁بن ا و ها ▁كان ▁ديال ▁تا م و دا ▁ ها ج رو


Special tokens in fastai start with letters xx, they are useful to help our model handle the shift from original text to our preprocessed text. For example, `xxunk` is used to replace the tokens that don't exist in our vocab, 
as it can be useful to help our model learn to deal with missing tokens.

In [ ]:
print(dls_lm.vocab[:20])

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxfld', 'xxrep', 'xxwrep', 'xxup', 'xxmaj', '▁', 'ة', 'و', 'ا', '▁ل', '▁ال', 'ش', 'م', 'ت', 'ر', 'ل']


In [ ]:
learn = language_model_learner(dls_lm, AWD_LSTM, 
    metrics=[accuracy, Perplexity()], pretrained=False)

In [ ]:
awd_lstm_lm_config

{'bidir': False,
 'emb_sz': 400,
 'embed_p': 0.02,
 'hidden_p': 0.15,
 'input_p': 0.25,
 'n_hid': 1152,
 'n_layers': 3,
 'out_bias': True,
 'output_p': 0.1,
 'pad_token': 1,
 'tie_weights': True,
 'weight_p': 0.2}

In [ ]:
learn.fit_one_cycle(n_epoch=50, lr_max=1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,5.528187,5.505463,0.046693,246.032379,01:56
1,5.495099,5.506052,0.045627,246.177307,01:58
2,5.488939,5.501623,0.046768,245.089432,01:58
3,4.782646,4.250841,0.186230,70.164375,01:59
4,4.076362,3.835729,0.241025,46.327183,01:57
5,3.668439,3.374904,0.316879,29.221491,01:58
6,3.298805,3.059468,0.367283,21.316210,01:58
7,3.063714,2.873476,0.395741,17.698427,01:58
8,2.949825,2.784125,0.409532,16.185646,01:57
9,2.845166,2.724165,0.420498,15.243683,01:58


In [ ]:
learn.save(model_path/'darija_lm')

NameError: ignored

In [ ]:
def decoder(sentence):
  s = ''.join(sentence)
  return s.split('▁')

In [ ]:
text = 'رسام صانع طباعة'
n_toks = 100
n_sentences = 2
preds = [learn.predict(text, n_toks, temperature=0.75, decoder=decoder)
        for _ in range(n_sentences)]

In [ ]:
preds

[' xxbos رسام صانع طباعة كبار كلوب وكيفاش ربحات لمريض ديال لملحون لفيلم ديال ميدلت تزاد عام لقبيلة أبريل كان كاتب مكسيك بوكوس اني كان هو هذا نهار دوايات تسمات حيزب واحد رموز ديال مينال ضد لبوطولة لي كاتهبط لملك محمد ساد',
 ' xxbos رسام صانع طباعة صدرات ديال لكوميديا پريمو جيري نوپنز أرنج هاريمي ماري سيكو هو أكبر عدد ديال الناس من الناحية التسعينات الجينيرالية الجوامع اللي فيها بزاف ديال الناس فعام ديال التعليم والتلت واللي دارو بزاف دلمنشيدات لموغمارية اللي عندها ف']

In [ ]:
text = 'رسام صانع طباعة'
n_toks = 100
n_sentences = 2
preds = [learn.predict(text, n_toks, temperature=0.75)
        for _ in range(n_sentences)]

In [ ]:
preds

['▁xxbos ▁ ر س ام ▁ ص ان ع ▁ط ب اع ة ▁م ك ع ب ة ▁ت م ▁ ك ل ▁ سم يت و ▁لع ما لق ة ▁م ق اط ع ة ▁م وري طان ية ▁ت ز اد ت ▁عام ▁ ج زي ر ة ▁يو ني و ▁معروف ة ▁بال ش ع ر ية ▁ديال ▁ت ور ة ▁ديال ▁ ص غ ر ها ▁ يت ز اد ▁وت أ س س ات ▁هي ▁ق نا ة ▁ ش ع بي ة ▁ پ ور ت ▁ ري ت ش ار د ▁جو ن ▁بن ▁أ ح م د ▁في ▁ ض هر ▁واحد ▁من ▁ خد',
 '▁xxbos ▁ ر س ام ▁ ص ان ع ▁ط ب اع ة ▁كت ر ▁من ▁م ليون ▁واحد ▁كاي ت م يز ▁أو لي پ ▁ ين و ية ▁لك ات يب ▁ح ض ة ▁كبير ة ▁بزاف ▁لب دي ة ▁لق ص ة ▁ديال ▁ ض و ▁الن قل ▁الس ع ود ية ▁بين ما ▁ ست ة ▁د ال ن ها ر ▁م لف ▁jp g ▁ت ص غ ير ▁ب ك ▁ ص غ ير ة ▁فال ت اري خ ▁ال ث اني ▁أ ما زي غ ي ▁ديال ▁ ش م ال ▁د ال م ص ا دير ▁ v i e w ▁ b ar b la']